# SageMaker Pipeline ends at RegisterModel with PendingManualApproval (finite run; it should not “wait days”).

Then EventBridge + Lambda do the “human-in-the-loop” part:

Notify approvers via SNS email when a new model package is pending.

Auto-deploy a serverless endpoint when the model package becomes Approved.

This matches how SageMaker Model Registry is designed: model packages have an approval status like PendingManualApproval / Approved, and you can view/update it via Studio/Boto3.

**Architecture (what you will build)**

**Event 1: New model package pending approval**

EventBridge Rule (Model Package state change)
- Lambda “notify”
- SNS topic → email to approvers

**Event 2: Model package approved**

EventBridge Rule (approval status becomes Approved)
- Lambda “deploy”
- create/update serverless endpoint

**You can implement this as:**

- 2 separate Lambdas + 2 rules (clearer), or

- 1 Lambda + 1 rule (handles both Pending and Approved).

*I’ll show the 1 Lambda approach (simpler to maintain).*

In [1]:
import json, time, textwrap, zipfile, os, io
import boto3
import sagemaker

region = boto3.Session().region_name
sts = boto3.client("sts", region_name=region)
account_id = sts.get_caller_identity()["Account"]

print("Region:", region)
print("Account:", account_id)

sm_role_arn = sagemaker.get_execution_role()
print("SageMaker execution role ARN (use as model role):", sm_role_arn)

sns_client = boto3.client("sns", region_name=region)
iam = boto3.client("iam", region_name=region)
lambda_client = boto3.client("lambda", region_name=region)
events = boto3.client("events", region_name=region)
sagemaker_client = boto3.client("sagemaker", region_name=region)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Region: us-east-1
Account: 423623839320
SageMaker execution role ARN (use as model role): arn:aws:iam::423623839320:role/service-role/SageMaker-ExecutionRole-20250705T232334


# Create SNS topic + email subscription

In [2]:
SNS_TOPIC_NAME = "retail-demand-model-approvals"
APPROVER_EMAIL = "smoothich@gmail.com"  # <-- change this

topic_arn = sns_client.create_topic(Name=SNS_TOPIC_NAME)["TopicArn"]
print("SNS Topic ARN:", topic_arn)

sub = sns_client.subscribe(
    TopicArn=topic_arn,
    Protocol="email",
    Endpoint=APPROVER_EMAIL,
)
print("Subscription created:", sub["SubscriptionArn"])
print("✅ Check your email and CONFIRM the subscription (required).")


SNS Topic ARN: arn:aws:sns:us-east-1:423623839320:retail-demand-model-approvals
Subscription created: pending confirmation
✅ Check your email and CONFIRM the subscription (required).


In [3]:
subs = sns_client.list_subscriptions_by_topic(TopicArn=topic_arn)["Subscriptions"]
subs


[{'SubscriptionArn': 'arn:aws:sns:us-east-1:423623839320:retail-demand-model-approvals:1b2dddc6-97a9-43fb-bcaa-af37357515a6',
  'Owner': '423623839320',
  'Protocol': 'email',
  'Endpoint': 'smoothich@gmail.com',
  'TopicArn': 'arn:aws:sns:us-east-1:423623839320:retail-demand-model-approvals'}]

# Define deployment settings (serverless endpoint)

In [4]:
MODEL_PACKAGE_GROUP_NAME = "retail-demand-model-group"  # <-- your model package group
ENDPOINT_NAME = "retail-demand-xgb-serverless-prod"     # <-- stable endpoint name for prod

SERVERLESS_MEMORY_SIZE = 4096
SERVERLESS_MAX_CONCURRENCY = 10


# Create IAM role for Lambda (execution role)

In [5]:
LAMBDA_ROLE_NAME = "retail-demand-approval-deployer-role"
LAMBDA_ROLE_ARN = None

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "lambda.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

try:
    resp = iam.create_role(
        RoleName=LAMBDA_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Lambda role for SNS notify + deploy SageMaker endpoint on approval",
    )
    LAMBDA_ROLE_ARN = resp["Role"]["Arn"]
    print("Created role:", LAMBDA_ROLE_ARN)
except iam.exceptions.EntityAlreadyExistsException:
    LAMBDA_ROLE_ARN = iam.get_role(RoleName=LAMBDA_ROLE_NAME)["Role"]["Arn"]
    print("Role already exists:", LAMBDA_ROLE_ARN)

# Attach basic CloudWatch Logs permissions
iam.attach_role_policy(
    RoleName=LAMBDA_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
)
print("Attached AWSLambdaBasicExecutionRole")


Created role: arn:aws:iam::423623839320:role/retail-demand-approval-deployer-role
Attached AWSLambdaBasicExecutionRole


In [25]:
inline_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "SageMakerRegistryRead",
      "Effect": "Allow",
      "Action": ["sagemaker:DescribeModelPackage"],
      "Resource": "*"
    },
    {
      "Sid": "SageMakerDeploy",
      "Effect": "Allow",
      "Action": [
        "sagemaker:CreateModel",
        "sagemaker:CreateEndpointConfig",
        "sagemaker:CreateEndpoint",
        "sagemaker:UpdateEndpoint",
        "sagemaker:DescribeEndpoint",
        "sagemaker:AddTags" 
      ],
      "Resource": "*"
    },
    {
      "Sid": "AllowPassRoleToSageMaker",
      "Effect": "Allow",
      "Action": "iam:PassRole",
      "Resource": sm_role_arn
    },
    {
      "Sid": "SNSPublish",
      "Effect": "Allow",
      "Action": "sns:Publish",
      "Resource": topic_arn
    }
  ]
}

iam.put_role_policy(
    RoleName=LAMBDA_ROLE_NAME,
    PolicyName="retail-demand-approval-deployer-inline",
    PolicyDocument=json.dumps(inline_policy),
)
print("Inline policy attached.")
time.sleep(20)


Inline policy attached.


# Create Lambda function package (zip) in-memory

In [26]:
LAMBDA_FUNCTION_NAME = "retail-demand-approval-notify-deploy"
handler_code = r'''
import os
import json
import time
import boto3
from botocore.exceptions import ClientError

region = os.environ.get("AWS_REGION", "us-east-1")
sm = boto3.client("sagemaker", region_name=region)
sns = boto3.client("sns", region_name=region)

SNS_TOPIC_ARN = os.environ["SNS_TOPIC_ARN"]
ENDPOINT_NAME = os.environ.get("ENDPOINT_NAME", "retail-demand-xgb-serverless-prod")
SAGEMAKER_MODEL_ROLE_ARN = os.environ["SAGEMAKER_MODEL_ROLE_ARN"]
SERVERLESS_MEMORY_SIZE = int(os.environ.get("SERVERLESS_MEMORY_SIZE", "4096"))
SERVERLESS_MAX_CONCURRENCY = int(os.environ.get("SERVERLESS_MAX_CONCURRENCY", "10"))
EXPECTED_GROUP = os.environ.get("MODEL_PACKAGE_GROUP_NAME", "").strip() or None

VARIANT_NAME = "AllTraffic"

def _extract_model_package_arn(event: dict) -> str:
    detail = event.get("detail", {})
    return (
        detail.get("ModelPackageArn")
        or detail.get("ModelPackageName")
        or detail.get("modelPackageArn")
        or detail.get("modelPackageName")
        or ""
    )

def _safe_name(s: str, max_len: int = 63) -> str:
    s = s.replace(":", "-").replace("/", "-").replace("_", "-")
    return s[:max_len]

def notify_pending(mp_arn: str, desc: dict):
    group = desc.get("ModelPackageGroupName")
    version = desc.get("ModelPackageVersion")
    approval = desc.get("ModelApprovalStatus")
    status = desc.get("ModelPackageStatus")

    msg = (
        "A new SageMaker Model Package is pending manual approval.\n\n"
        f"Group: {group}\n"
        f"Version: {version}\n"
        f"ApprovalStatus: {approval}\n"
        f"PackageStatus: {status}\n"
        f"ModelPackageArn: {mp_arn}\n\n"
        "Action: Please review & set ApprovalStatus = Approved in SageMaker Model Registry."
    )

    sns.publish(
        TopicArn=SNS_TOPIC_ARN,
        Subject="[SageMaker] Model pending approval",
        Message=msg,
    )
    print("[SNS] Sent pending-approval notification.")

def deploy_serverless(mp_arn: str, desc: dict):
    group = desc.get("ModelPackageGroupName")
    version = desc.get("ModelPackageVersion")

    ts = time.strftime("%Y%m%d-%H%M%S", time.gmtime())
    base = _safe_name(ENDPOINT_NAME, 40)
    model_name = _safe_name(f"{base}-m-{version}-{ts}")
    cfg_name = _safe_name(f"{base}-cfg-{version}-{ts}")

    print(f"[DEPLOY] Creating model: {model_name}")
    sm.create_model(
        ModelName=model_name,
        ExecutionRoleArn=SAGEMAKER_MODEL_ROLE_ARN,
        PrimaryContainer={"ModelPackageName": mp_arn},
        Tags=[
            {"Key": "ModelPackageArn", "Value": mp_arn},
            {"Key": "ModelPackageGroupName", "Value": str(group or "")},
            {"Key": "ModelPackageVersion", "Value": str(version or "")},
        ],
    )

    print(f"[DEPLOY] Creating endpoint config (serverless): {cfg_name}")
    sm.create_endpoint_config(
        EndpointConfigName=cfg_name,
        ProductionVariants=[
            {
                "VariantName": VARIANT_NAME,
                "ModelName": model_name,
                "InitialVariantWeight": 1.0,
                "ServerlessConfig": {
                    "MemorySizeInMB": SERVERLESS_MEMORY_SIZE,
                    "MaxConcurrency": SERVERLESS_MAX_CONCURRENCY,
                },
            }
        ],
    )

    # Create or Update endpoint
    try:
        sm.describe_endpoint(EndpointName=ENDPOINT_NAME)
        print(f"[DEPLOY] Endpoint exists. Updating endpoint: {ENDPOINT_NAME}")
        sm.update_endpoint(EndpointName=ENDPOINT_NAME, EndpointConfigName=cfg_name)
    except ClientError as e:
        code = e.response["Error"]["Code"]
        if code in ("ValidationException", "ResourceNotFoundException"):
            print(f"[DEPLOY] Endpoint not found. Creating endpoint: {ENDPOINT_NAME}")
            sm.create_endpoint(EndpointName=ENDPOINT_NAME, EndpointConfigName=cfg_name)
        else:
            raise

    print("[DEPLOY] Deployment request submitted (async).")

def lambda_handler(event, context):
    print("Event:", json.dumps(event))

    mp_arn = _extract_model_package_arn(event)
    if not mp_arn:
        return {"action": "skip", "reason": "no_model_package_arn"}

    desc = sm.describe_model_package(ModelPackageName=mp_arn)

    group = desc.get("ModelPackageGroupName")
    approval = desc.get("ModelApprovalStatus")
    status = desc.get("ModelPackageStatus")

    print(f"[REGISTRY] group={group} approval={approval} status={status} arn={mp_arn}")

    if EXPECTED_GROUP and group != EXPECTED_GROUP:
        print(f"[SKIP] ModelPackageGroupName != expected ({EXPECTED_GROUP})")
        return {"action": "skip", "reason": "wrong_group", "group": group}

    if approval == "PendingManualApproval":
        notify_pending(mp_arn, desc)
        return {"action": "notified_pending", "model_package_arn": mp_arn}

    if approval == "Approved" and status == "Completed":
        deploy_serverless(mp_arn, desc)
        return {"action": "deploy_started", "endpoint_name": ENDPOINT_NAME, "model_package_arn": mp_arn}

    return {"action": "skip", "approval": approval, "status": status}
'''

# Create zip in-memory
buf = io.BytesIO()
with zipfile.ZipFile(buf, "w", zipfile.ZIP_DEFLATED) as z:
    z.writestr("lambda_function.py", handler_code)
buf.seek(0)

lambda_zip_bytes = buf.read()
len(lambda_zip_bytes)


1813

# Create or update Lambda function

In [8]:
env_vars = {
    "SNS_TOPIC_ARN": topic_arn,
    "ENDPOINT_NAME": ENDPOINT_NAME,
    "SAGEMAKER_MODEL_ROLE_ARN": sm_role_arn,
    "SERVERLESS_MEMORY_SIZE": str(SERVERLESS_MEMORY_SIZE),
    "SERVERLESS_MAX_CONCURRENCY": str(SERVERLESS_MAX_CONCURRENCY),
    "MODEL_PACKAGE_GROUP_NAME": MODEL_PACKAGE_GROUP_NAME,
}

try:
    resp = lambda_client.create_function(
        FunctionName=LAMBDA_FUNCTION_NAME,
        Runtime="python3.11",
        Role=LAMBDA_ROLE_ARN,
        Handler="lambda_function.lambda_handler",
        Code={"ZipFile": lambda_zip_bytes},
        Timeout=60,
        MemorySize=256,
        Environment={"Variables": env_vars},
    )
    lambda_arn = resp["FunctionArn"]
    print("Created Lambda:", lambda_arn)
except lambda_client.exceptions.ResourceConflictException:
    # Update code + config
    lambda_client.update_function_code(
        FunctionName=LAMBDA_FUNCTION_NAME,
        ZipFile=lambda_zip_bytes,
        Publish=True,
    )
    lambda_client.update_function_configuration(
        FunctionName=LAMBDA_FUNCTION_NAME,
        Runtime="python3.11",
        Role=LAMBDA_ROLE_ARN,
        Handler="lambda_function.lambda_handler",
        Timeout=60,
        MemorySize=256,
        Environment={"Variables": env_vars},
    )
    lambda_arn = lambda_client.get_function(FunctionName=LAMBDA_FUNCTION_NAME)["Configuration"]["FunctionArn"]
    print("Updated Lambda:", lambda_arn)

# Wait for Lambda to be active
time.sleep(10)


Created Lambda: arn:aws:lambda:us-east-1:423623839320:function:retail-demand-approval-notify-deploy


In [ ]:
import json, time
import boto3

region = boto3.Session().region_name
iam = boto3.client("iam", region_name=region)

LAMBDA_ROLE_NAME = "retail-demand-approval-deployer-role"
POLICY_NAME = "retail-demand-approval-deployer-inline"

# Get existing policy document
doc = iam.get_role_policy(RoleName=LAMBDA_ROLE_NAME, PolicyName=POLICY_NAME)["PolicyDocument"]

# Add sagemaker:AddTags to the SageMakerDeploy statement
for stmt in doc["Statement"]:
    if stmt.get("Sid") == "SageMakerDeploy":
        actions = stmt["Action"]
        if isinstance(actions, str):
            actions = [actions]
        if "sagemaker:AddTags" not in actions:
            actions.append("sagemaker:AddTags")
        stmt["Action"] = actions

# Put policy back
iam.put_role_policy(
    RoleName=LAMBDA_ROLE_NAME,
    PolicyName=POLICY_NAME,
    PolicyDocument=json.dumps(doc),
)

print("✅ Updated inline policy. Added sagemaker:AddTags")
print("Waiting IAM propagation...")
time.sleep(20)


# Create EventBridge rule → trigger Lambda on model package state changes

In [9]:
RULE_NAME = "retail-demand-model-package-state-change"

event_pattern = {
  "source": ["aws.sagemaker"],
  "detail-type": ["SageMaker Model Package State Change"],
  "detail": {
      "ModelPackageGroupName": [MODEL_PACKAGE_GROUP_NAME],
      "ModelApprovalStatus": ["PendingManualApproval", "Approved"]
  }
}

rule_arn = events.put_rule(
    Name=RULE_NAME,
    EventPattern=json.dumps(event_pattern),
    State="ENABLED",
    Description="Notify on PendingManualApproval and deploy on Approved for retail demand models",
)["RuleArn"]

print("Rule ARN:", rule_arn)

events.put_targets(
    Rule=RULE_NAME,
    Targets=[{
        "Id": "LambdaTarget",
        "Arn": lambda_arn
    }]
)

# Allow EventBridge to invoke Lambda
statement_id = "AllowEventBridgeInvoke"
try:
    lambda_client.add_permission(
        FunctionName=LAMBDA_FUNCTION_NAME,
        StatementId=statement_id,
        Action="lambda:InvokeFunction",
        Principal="events.amazonaws.com",
        SourceArn=rule_arn,
    )
    print("Added Lambda invoke permission for EventBridge")
except lambda_client.exceptions.ResourceConflictException:
    print("Lambda permission already exists (ok)")


Rule ARN: arn:aws:events:us-east-1:423623839320:rule/retail-demand-model-package-state-change
Added Lambda invoke permission for EventBridge


✅ Now your automation is live.

# Manual test (invoke Lambda with a real model_package_arn)

In [17]:
group_name = "retail-demand-model-group"

# List latest model package versions in the group
resp = sagemaker_client.list_model_packages(
    ModelPackageGroupName=group_name,
    SortBy="CreationTime",
    SortOrder="Descending",
    MaxResults=5,
)

for i, p in enumerate(resp["ModelPackageSummaryList"], 1):
    print(i, p["ModelPackageArn"], p["ModelApprovalStatus"], p["ModelPackageVersion"])

# Pick the latest one (or choose one you want)
model_package_arn = resp["ModelPackageSummaryList"][0]["ModelPackageArn"]
print("\nUse this model_package_arn:", model_package_arn)


1 arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/8 PendingManualApproval 8
2 arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/7 PendingManualApproval 7
3 arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/6 Approved 6
4 arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/5 Approved 5
5 arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/4 Approved 4

Use this model_package_arn: arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/8


Test PendingManualApproval Model

In [18]:
# Put a real model package arn here
test_model_package_arn = "arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/8"  # <-- change

test_event = {
  "detail-type": "SageMaker Model Package State Change",
  "source": "aws.sagemaker",
  "detail": {
      "ModelPackageArn": test_model_package_arn,
      "ModelPackageGroupName": MODEL_PACKAGE_GROUP_NAME,
      # simulate the pending event:
      "ModelApprovalStatus": "PendingManualApproval",
  }
}

resp = lambda_client.invoke(
    FunctionName=LAMBDA_FUNCTION_NAME,
    InvocationType="RequestResponse",
    Payload=json.dumps(test_event).encode("utf-8"),
)
print(resp["Payload"].read().decode("utf-8"))


{"action": "notified_pending", "model_package_arn": "arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/8"}


Test Approve Model

In [22]:
sagemaker_client.update_model_package(
    ModelPackageArn=test_model_package_arn,
    ModelApprovalStatus="Approved",
    ApprovalDescription="Approved for serverless deployment via automation test."
)
print("Approved:", test_model_package_arn)


Approved: arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/8


In [27]:
test_event = {
  "detail-type": "SageMaker Model Package State Change",
  "source": "aws.sagemaker",
  "detail": {
      "ModelPackageArn": test_model_package_arn,
      "ModelPackageGroupName": "retail-demand-model-group",
      "ModelApprovalStatus": "Approved",
  }
}

resp = lambda_client.invoke(
    FunctionName=LAMBDA_FUNCTION_NAME,
    InvocationType="RequestResponse",
    Payload=json.dumps(test_event).encode("utf-8"),
)
print(resp["Payload"].read().decode("utf-8"))


{"action": "deploy_started", "endpoint_name": "retail-demand-xgb-serverless-prod", "model_package_arn": "arn:aws:sagemaker:us-east-1:423623839320:model-package/retail-demand-model-group/8"}


In [30]:
desc = sagemaker_client.describe_endpoint(EndpointName="retail-demand-xgb-serverless-prod")
print(desc["EndpointStatus"], desc.get("FailureReason", ""))


InService 


# Test inference

In [36]:
import pandas as pd
import numpy as np

FEATURE_ORDER = [
    "store_id",
    "is_weekend",
    "is_holiday",
    "max_temp_c",
    "rainfall_mm",
    "is_hot_day",
    "is_rainy_day",
    "base_price",
    "discount_pct",
    "is_promo",
    "final_price",
    "year",
    "month",
    "day",
    "day_of_year",
    "day_of_week_index",
    "discount_amount",
    "is_promo_or_holiday",
]


def df_to_csv_payload(df: pd.DataFrame) -> str:
    """Reorder columns to FEATURE_ORDER and convert to headerless CSV string."""
    missing = [c for c in FEATURE_ORDER if c not in df.columns]
    if missing:
        raise ValueError(f"Missing features in input DataFrame: {missing}")

    X = df[FEATURE_ORDER].astype(np.float32).to_numpy()
    lines = [",".join(map(str, row)) for row in X]
    return "\n".join(lines)

# Example: build a one-row DataFrame (fake data)
df_request = pd.DataFrame([{
    "store_id": 1,
    "is_weekend": 0,
    "is_holiday": 0,
    "max_temp_c": 30.5,
    "rainfall_mm": 2.0,
    "is_hot_day": 1,
    "is_rainy_day": 0,
    "base_price": 12.5,
    "discount_pct": 0.1,
    "is_promo": 1,
    "final_price": 11.25,
    "year": 2024,
    "month": 2,
    "day": 15,
    "day_of_year": 46,
    "day_of_week_index": 3,
    "discount_amount": 1.25,
    "is_promo_or_holiday": 1,
}])

runtime_sm = boto3.client("sagemaker-runtime", region_name=region)

# Build payload string using the same FEATURE_ORDER / df_to_csv_payload
payload = df_to_csv_payload(df_request)

response = runtime_sm.invoke_endpoint(
    EndpointName=f"{desc["EndpointName"]}",
    ContentType="text/csv",
    Body=payload.encode("utf-8"),
)

body = response["Body"].read().decode("utf-8")
print("Raw body:", body)



Raw body: 88.05634307861328

